# Data Cleaning Notebook - Models 1 & 2

**Medicare DeSYNPuf Data:
2009 Inpatient Claims for Risk Adjustment Modeling and Correspondence Analysis**

HDS 823 Final Project: Advanced Statistics in Healthcare
Kyle P. Rasku RN BSN

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

#For standardizing variables
import sklearn.preprocessing as pre


Starting with the Elixhauser Score files.

By averaging, produce 1 score for each DeSYNPuf ID.

Note: The Elixhauser Score files were producted by Dhara Kapoor MSPT, using SAS code provided by CMS.

In [2]:
elix = pd.read_csv("/home/kylier/python/DS/data/PUF/IPANDOP_FIPS_ELIX_ALL.csv")

In [3]:
elix.shape

(1332236, 57)

In [4]:
for c in elix.columns:
    print(c)

Unnamed: 0
Unnamed: 1
DESYNPUF_ID
BENE_SEX_IDENT_CD
BENE_RACE_CD
SP_STATE_CODE
BENE_COUNTY_CD
FIPS_COUNTY_CODE
FULL_FIPS_CODE
FIPS_STATE_CODE
ICD9_DGNS_CD_1
ICD9_DGNS_CD_2
ICD9_DGNS_CD_3
ICD9_DGNS_CD_4
ICD9_DGNS_CD_5
ICD9_DGNS_CD_6
ICD9_DGNS_CD_7
ICD9_DGNS_CD_8
ICD9_DGNS_CD_9
ICD9_DGNS_CD_10
CLAIM_TYPE
BENE_AGE
YEAR
CHF
Car_Arrhythmia
Valve_Disease
Pulm_Cir_Disorder
PVD
HT_Uncomp
HT_Comp
Paralysis
Neuro_disorder_other
CPD
DM_Uncomp
DM_Comp
Hypo_thyroid
Renal_Failure
Liver_Disease
Peptic_Ulcer_WO_bleeding
AIDS_HIV
Lymphoma
Metastatic_Cancer
Tumor_WO_Metastasis
RA
Coagulopathy
Obesity
Weight_Loss
Fluid_Electrolyte_Disorder
Blood_Loss_Anemia
Deficiency_Anemia
Alcohol_Abuse
Drug_Abuse
Psychoses
Depression
i
TOT_GRP
elx_WEIGHT_SCORE


Drop the first 2 columns (created when the csv file was written out)

Show nulls

In [5]:
elix.drop(columns=elix.columns[0], axis=1, inplace=True)
elix.drop(columns=elix.columns[0], axis=1, inplace=True)
elix.isnull().sum() 

DESYNPUF_ID                         0
BENE_SEX_IDENT_CD                   0
BENE_RACE_CD                        0
SP_STATE_CODE                       0
BENE_COUNTY_CD                      0
FIPS_COUNTY_CODE                14272
FULL_FIPS_CODE                  14272
FIPS_STATE_CODE                 14272
ICD9_DGNS_CD_1                 187985
ICD9_DGNS_CD_2                 576088
ICD9_DGNS_CD_3                 822826
ICD9_DGNS_CD_4                 986948
ICD9_DGNS_CD_5                1097620
ICD9_DGNS_CD_6                1155237
ICD9_DGNS_CD_7                1192973
ICD9_DGNS_CD_8                1219278
ICD9_DGNS_CD_9                1239874
ICD9_DGNS_CD_10               1322764
CLAIM_TYPE                     187912
BENE_AGE                       187912
YEAR                                0
CHF                                 0
Car_Arrhythmia                      0
Valve_Disease                       0
Pulm_Cir_Disorder                   0
PVD                                 0
HT_Uncomp   

In [10]:
elix["ELIX_GRP_TOTAL"] = elix["TOT_GRP"].copy()
elix["ELIX_SCORE"] = elix["elx_WEIGHT_SCORE"].copy()

In [11]:
elix = elix.drop(elix.columns[[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 
                                   24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 
                                   43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]], axis=1, errors="ignore")

# Subset only 2009 data
elix = elix[elix.YEAR==2009]
elix.head()

,DESYNPUF_ID,SP_STATE_CODE,BENE_COUNTY_CD,YEAR,ELIX_GRP_TOTAL,ELIX_SCORE
363874,0000438E79D01BEA,10,280,2009,0,0
363875,00010E7BEA69142C,8,10,2009,2,3
363876,00010E7BEA69142C,8,10,2009,0,0
363877,00010E7BEA69142C,8,10,2009,0,0
363878,00015BF6509E1DF7,34,590,2009,0,0


In [12]:
# Aggregate Scores by DeSYNPuf ID 
elix = elix.groupby(["DESYNPUF_ID"]).aggregate({"ELIX_GRP_TOTAL":"mean",
                                                "ELIX_SCORE":"mean"})

elix.columns = ["MEAN_ELIX_GRP_TOT", "MEAN_ELIX_SCORE"]
elix = elix.reset_index()
elix.head()


,DESYNPUF_ID,MEAN_ELIX_GRP_TOT,MEAN_ELIX_SCORE
0,0000438E79D01BEA,0.000000,0.0
1,00010E7BEA69142C,0.666667,1.0
2,00015BF6509E1DF7,0.000000,0.0
3,000240D599ED789C,1.000000,2.0
4,0003E73AFBA840A8,0.000000,0.0


In [13]:
# Load 2009 IP Data
desynpuf = pd.read_csv("/home/kylier/python/DS/data/PUF/IP2009.csv")

In [14]:
desynpuf.head()

,Unnamed: 0,Unnamed: 1,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,...,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,BENE_AGE,YEAR
0,2009,0,0000438E79D01BEA,1937-05-01,NaN,1,1,0,10,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,2009,1,00010E7BEA69142C,1938-04-01,2009-11-01,2,1,0,8,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
2,2009,2,00015BF6509E1DF7,1934-05-01,NaN,2,1,0,34,590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009
3,2009,3,000240D599ED789C,1942-04-01,NaN,2,2,0,52,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,2009
4,2009,4,0003E73AFBA840A8,1941-09-01,NaN,2,1,0,33,370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009


In [15]:
desynpuf.drop(columns=desynpuf.columns[0], axis=1, inplace=True)
desynpuf.drop(columns=desynpuf.columns[0], axis=1, inplace=True)
desynpuf.isnull().sum() 

DESYNPUF_ID               0
BENE_BIRTH_DT             0
BENE_DEATH_DT        118801
BENE_SEX_IDENT_CD         0
BENE_RACE_CD              0
                      ...  
HCPCS_CD_43          120705
HCPCS_CD_44          120705
HCPCS_CD_45          120705
BENE_AGE              95832
YEAR                      0
Length: 121, dtype: int64

In [16]:
desynpuf.shape

(120705, 121)

In [17]:
for c in desynpuf.columns:
    print(c)

DESYNPUF_ID
BENE_BIRTH_DT
BENE_DEATH_DT
BENE_SEX_IDENT_CD
BENE_RACE_CD
BENE_ESRD_IND
SP_STATE_CODE
BENE_COUNTY_CD
BENE_HI_CVRAGE_TOT_MONS
BENE_SMI_CVRAGE_TOT_MONS
BENE_HMO_CVRAGE_TOT_MONS
PLAN_CVRG_MOS_NUM
SP_ALZHDMTA
SP_CHF
SP_CHRNKIDN
SP_CNCR
SP_COPD
SP_DEPRESSN
SP_DIABETES
SP_ISCHMCHT
SP_OSTEOPRS
SP_RA_OA
SP_STRKETIA
MEDREIMB_IP
BENRES_IP
PPPYMT_IP
MEDREIMB_OP
BENRES_OP
PPPYMT_OP
MEDREIMB_CAR
BENRES_CAR
PPPYMT_CAR
FIPS_COUNTY_CODE
FULL_FIPS_CODE
CBSA
CBSA_NAME
FIPS_STATE_CODE
CLM_ID
SEGMENT
CLM_FROM_DT
CLM_THRU_DT
PRVDR_NUM
CLM_PMT_AMT
NCH_PRMRY_PYR_CLM_PD_AMT
AT_PHYSN_NPI
OP_PHYSN_NPI
OT_PHYSN_NPI
CLM_ADMSN_DT
ADMTNG_ICD9_DGNS_CD
CLM_PASS_THRU_PER_DIEM_AMT
NCH_BENE_IP_DDCTBL_AMT
NCH_BENE_PTB_DDCTBL_AMT
NCH_BENE_PTA_COINSRNC_LBLTY_AM
NCH_BENE_PTB_COINSRNC_AMT
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM
CLM_UTLZTN_DAY_CNT
NCH_BENE_DSCHRG_DT
CLM_DRG_CD
ICD9_DGNS_CD_1
ICD9_DGNS_CD_2
ICD9_DGNS_CD_3
ICD9_DGNS_CD_4
ICD9_DGNS_CD_5
ICD9_DGNS_CD_6
ICD9_DGNS_CD_7
ICD9_DGNS_CD_8
ICD9_DGNS_CD_9
ICD9_DGNS_CD

## Clean Age

Clean BENE_AGE by calculating rough age of all beneficiaries for the year 2009, regardless of claim status.

In 2009, 95,832 beneficiaries had no claims in this claim data set.

In [18]:
# NOTE: ALWAYS USE .copy(), otherwise face the WRATH of SETTING WITH COPY WARNINGS!!!!!
null_age = desynpuf[desynpuf["BENE_AGE"].isnull()].copy()
null_age.shape

(95832, 121)

In [19]:
import datetime as dt

null_age.loc[:, ("START_DATE")] = pd.to_datetime("20091231")
null_age[['BENE_BIRTH_DT']] = pd.to_datetime(null_age["BENE_BIRTH_DT"], format="%Y-%m-%d")
null_age[['BENE_AGE']] = round((null_age["START_DATE"] - null_age["BENE_BIRTH_DT"]).dt.days / 365.2425)

null_age.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,BENE_AGE,YEAR,START_DATE
0,0000438E79D01BEA,1937-05-01,NaN,1,1,0,10,280,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,2009,2009-12-31
1,00010E7BEA69142C,1938-04-01,2009-11-01,2,1,0,8,10,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,2009,2009-12-31
2,00015BF6509E1DF7,1934-05-01,NaN,2,1,0,34,590,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,2009,2009-12-31
4,0003E73AFBA840A8,1941-09-01,NaN,2,1,0,33,370,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2009,2009-12-31
5,00040FBA47F90D4F,1924-12-01,NaN,2,1,0,31,350,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,2009,2009-12-31


In [20]:
desynpuf.loc[desynpuf["BENE_AGE"].isnull(), "BENE_AGE"] = null_age["BENE_AGE"]
desynpuf.isnull().sum() 

DESYNPUF_ID               0
BENE_BIRTH_DT             0
BENE_DEATH_DT        118801
BENE_SEX_IDENT_CD         0
BENE_RACE_CD              0
                      ...  
HCPCS_CD_43          120705
HCPCS_CD_44          120705
HCPCS_CD_45          120705
BENE_AGE                  0
YEAR                      0
Length: 121, dtype: int64

## Group Age

For Correspondence Analysis (Model 2), categorical age data will be required.

I will create this using the age groups defined by CMS, except I will add one additional age group (75-84) because the distribution of the data skews high.

In [21]:
desynpuf.loc[((desynpuf["BENE_AGE"]>0) & (desynpuf["BENE_AGE"]<19)), "AGE_GROUP"] = "0-18"
desynpuf.loc[((desynpuf["BENE_AGE"]>18) & (desynpuf["BENE_AGE"]<45)), "AGE_GROUP"] = "19-44"
desynpuf.loc[((desynpuf["BENE_AGE"]>44) & (desynpuf["BENE_AGE"]<65)), "AGE_GROUP"] = "45-64"
desynpuf.loc[((desynpuf["BENE_AGE"]>64) & (desynpuf["BENE_AGE"]<75)), "AGE_GROUP"] = "65-74"
desynpuf.loc[((desynpuf["BENE_AGE"]>74) & (desynpuf["BENE_AGE"]<85)), "AGE_GROUP"] = "75-84"
desynpuf.loc[(desynpuf["BENE_AGE"]>84), "AGE_GROUP"] = "85+"


In [22]:
desynpuf["AGE_GROUP"].value_counts(normalize=True)

65-74    0.377118
75-84    0.300046
85+      0.168957
45-64    0.118496
19-44    0.035384
Name: AGE_GROUP, dtype: float64

## Binary Sex

Translate BENE_SEX_IDENT_CD into a binary variable: 0 for males, 1 for females

In [23]:
desynpuf["B_SEX"] = desynpuf["BENE_SEX_IDENT_CD"].replace(2, 0)
desynpuf["B_SEX"].value_counts(normalize=True)


0    0.557483
1    0.442517
Name: B_SEX, dtype: float64

## Chronic Conditions

Recode chronic conditions markers to 0 = No, 1 = Yes.

Keep these markers after summarizing.  The summaries will be used in the Risk Adjustment Model (RAM), but the binary markers will be used in the Correspondence Analysis.

In [24]:
desynpuf["SP_ESRD"] = desynpuf["BENE_ESRD_IND"].replace("Y", 1).astype(int)
desynpuf["SP_ESRD"] = desynpuf["SP_ESRD"].replace(0, 2).astype(int)
desynpuf["CLM_ADMSN_DT"] = pd.to_datetime(desynpuf["CLM_ADMSN_DT"], format="%Y%m%d")
desynpuf["NCH_BENE_DSCHRG_DT"] = pd.to_datetime(desynpuf["NCH_BENE_DSCHRG_DT"], format="%Y%m%d")

desynpuf.drop(["BENE_SEX_IDENT_CD", "BENE_ESRD_IND"], axis=1, inplace=True, errors="ignore")

In [25]:
# Re-code the conditions indicators as dummy variables

desynpuf["B_ALZHDMTA"] = desynpuf["SP_ALZHDMTA"].replace(2, 0).astype(int)
desynpuf["B_CHF"] = desynpuf["SP_CHF"].replace(2, 0).astype(int)
desynpuf["B_CNCR"] = desynpuf["SP_CNCR"].replace(2, 0).astype(int)
desynpuf["B_COPD"] = desynpuf["SP_COPD"].replace(2, 0).astype(int)
desynpuf["B_DEPRESSN"] = desynpuf["SP_DEPRESSN"].replace(2, 0).astype(int)
desynpuf["B_DIABETES"] = desynpuf["SP_DIABETES"].replace(2, 0).astype(int)
desynpuf["B_ISCHMCHT"] = desynpuf["SP_ISCHMCHT"].replace(2, 0).astype(int)
desynpuf["B_OSTEOPRS"] = desynpuf["SP_OSTEOPRS"].replace(2, 0).astype(int)
desynpuf["B_CHRNKIDN"] = desynpuf["SP_CHRNKIDN"].replace(2, 0).astype(int)
desynpuf["B_RA_OA"] = desynpuf["SP_RA_OA"].replace(2, 0).astype(int)
desynpuf["B_STRKETIA"] = desynpuf["SP_STRKETIA"].replace(2, 0).astype(int)
desynpuf["B_ESRD"] = desynpuf["SP_ESRD"].replace(2, 0).astype(int)


## Race & Ethnicity

Binarize and dummy code for race and ethnicity

In [26]:
lb = pre.LabelBinarizer()
BinarizedRace = pd.DataFrame(lb.fit_transform(desynpuf["BENE_RACE_CD"]), 
                             columns=["NH_WHITE", "AA_BLACK", "OTHER", "NW_HISPANIC"])
desynpuf = desynpuf.join(BinarizedRace)
desynpuf.drop(["BENE_RACE_CD"], axis=1, inplace=True, errors="ignore")
desynpuf.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,...,B_ISCHMCHT,B_OSTEOPRS,B_CHRNKIDN,B_RA_OA,B_STRKETIA,B_ESRD,NH_WHITE,AA_BLACK,OTHER,NW_HISPANIC
0,0000438E79D01BEA,1937-05-01,NaN,10,280,12,12,0,12,2,...,0,0,0,0,0,0,1,0,0,0
1,00010E7BEA69142C,1938-04-01,2009-11-01,8,10,12,12,12,12,1,...,0,0,0,0,0,0,1,0,0,0
2,00015BF6509E1DF7,1934-05-01,NaN,34,590,12,12,12,12,2,...,1,1,0,0,0,0,1,0,0,0
3,000240D599ED789C,1942-04-01,NaN,52,170,12,12,0,12,1,...,1,0,1,0,0,0,0,1,0,0
4,0003E73AFBA840A8,1941-09-01,NaN,33,370,0,0,0,0,2,...,0,0,0,0,0,0,1,0,0,0


Add Mean Elixhauser Scores for each Beneficiary (note: a score of zero does not necessarily indicate no conditions / health problems!)

ALSO: I would keep this to the specific Elix score for each admission, but I don't have the original code that created this output, and the output doesn't have the CLM_ID in it.  So, I can't join it back in on the level of the individual claim.  Mean scores will have to do.

In [27]:
elix['DESYNPUF_ID'] = elix['DESYNPUF_ID'].astype(str) 
desynpuf['DESYNPUF_ID'] = desynpuf['DESYNPUF_ID'].astype(str) 

desynpuf = pd.merge(desynpuf, elix, how='left', on='DESYNPUF_ID')
desynpuf.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,...,B_CHRNKIDN,B_RA_OA,B_STRKETIA,B_ESRD,NH_WHITE,AA_BLACK,OTHER,NW_HISPANIC,MEAN_ELIX_GRP_TOT,MEAN_ELIX_SCORE
0,0000438E79D01BEA,1937-05-01,NaN,10,280,12,12,0,12,2,...,0,0,0,0,1,0,0,0,0.000000,0.0
1,00010E7BEA69142C,1938-04-01,2009-11-01,8,10,12,12,12,12,1,...,0,0,0,0,1,0,0,0,0.666667,1.0
2,00015BF6509E1DF7,1934-05-01,NaN,34,590,12,12,12,12,2,...,0,0,0,0,1,0,0,0,0.000000,0.0
3,000240D599ED789C,1942-04-01,NaN,52,170,12,12,0,12,1,...,1,0,0,0,0,1,0,0,1.000000,2.0
4,0003E73AFBA840A8,1941-09-01,NaN,33,370,0,0,0,0,2,...,0,0,0,0,1,0,0,0,0.000000,0.0


## Codes per Claim

Count the number of ICD9 Codes per Claim

In [28]:
codes = desynpuf[["ICD9_DGNS_CD_1","ICD9_DGNS_CD_2","ICD9_DGNS_CD_3","ICD9_DGNS_CD_4","ICD9_DGNS_CD_5",
                  "ICD9_DGNS_CD_6","ICD9_DGNS_CD_7","ICD9_DGNS_CD_8","ICD9_DGNS_CD_9","ICD9_DGNS_CD_10",
                  "ICD9_PRCDR_CD_1","ICD9_PRCDR_CD_2","ICD9_PRCDR_CD_3","ICD9_PRCDR_CD_4","ICD9_PRCDR_CD_5",
                  "ICD9_PRCDR_CD_6"]]

desynpuf["CODES_COUNT"] = codes.count(axis=1)

# do not fill NaN values - NaNs are appropriate for patients without claims
# desynpuf["CODES_COUNT"].fillna(0, inplace=True)

## Procedures per Claim

HCPCS codes of the **Level I Type** are Procedure codes; these are 5 digit numbers.

Level II codes start with a letter.

Here, we will store the count of Level I / CPT codes for each beneficiary claim.

**NOTE: In this data (Inpatient Claims 2009) there are NO HCPCS CODES.**

In [30]:
# First, replace any L2-formatted codes with None / NULL

import re

procedures = desynpuf[["HCPCS_CD_1", "HCPCS_CD_2","HCPCS_CD_3","HCPCS_CD_4","HCPCS_CD_5","HCPCS_CD_6","HCPCS_CD_7",
                      "HCPCS_CD_8","HCPCS_CD_9","HCPCS_CD_10","HCPCS_CD_11","HCPCS_CD_12","HCPCS_CD_13",
                      "HCPCS_CD_14","HCPCS_CD_15","HCPCS_CD_16","HCPCS_CD_17","HCPCS_CD_18","HCPCS_CD_19",
                      "HCPCS_CD_20","HCPCS_CD_21","HCPCS_CD_22","HCPCS_CD_23","HCPCS_CD_24","HCPCS_CD_25",
                      "HCPCS_CD_26","HCPCS_CD_27","HCPCS_CD_28","HCPCS_CD_29","HCPCS_CD_30","HCPCS_CD_31",
                      "HCPCS_CD_32","HCPCS_CD_33","HCPCS_CD_34","HCPCS_CD_35","HCPCS_CD_36","HCPCS_CD_37",
                      "HCPCS_CD_38","HCPCS_CD_39","HCPCS_CD_40","HCPCS_CD_41","HCPCS_CD_42","HCPCS_CD_43",
                      "HCPCS_CD_44","HCPCS_CD_45"]]


procedures.head()

,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,HCPCS_CD_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,...,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Drop rows with NO procedures: new rowcount = 741,581
procedures = procedures.dropna(how="all")


# Drop columns with NO codes: new column count = 44 (0-43)
procedures = procedures.dropna(axis=1, how="all")

# Cast all to type "string"
procedures = procedures.astype("string")

# Replace L2 codes with NULL / NaN because we do not count them as procedures.
procedures = procedures.replace(r"([A-Z]\d{4})", np.nan,regex=True)


In [32]:
# Now that we know we only have L1 / CPT codes, count them for each beneficiary claim

desynpuf["HCPCS_COUNT"] = procedures.count(axis=1)

# do not fill NaN values - NaNs are appropriate for patients without claims
# desynpuf["HCPCS_COUNT"].fillna(0, inplace=True)

In [33]:
desynpuf.head(5)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,...,B_STRKETIA,B_ESRD,NH_WHITE,AA_BLACK,OTHER,NW_HISPANIC,MEAN_ELIX_GRP_TOT,MEAN_ELIX_SCORE,CODES_COUNT,HCPCS_COUNT
0,0000438E79D01BEA,1937-05-01,NaN,10,280,12,12,0,12,2,...,0,0,1,0,0,0,0.000000,0.0,0,NaN
1,00010E7BEA69142C,1938-04-01,2009-11-01,8,10,12,12,12,12,1,...,0,0,1,0,0,0,0.666667,1.0,0,NaN
2,00015BF6509E1DF7,1934-05-01,NaN,34,590,12,12,12,12,2,...,0,0,1,0,0,0,0.000000,0.0,0,NaN
3,000240D599ED789C,1942-04-01,NaN,52,170,12,12,0,12,1,...,0,0,0,1,0,0,1.000000,2.0,15,NaN
4,0003E73AFBA840A8,1941-09-01,NaN,33,370,0,0,0,0,2,...,0,0,1,0,0,0,0.000000,0.0,0,NaN


In [34]:
desynpuf.shape

(120705, 141)

In [35]:
for c in desynpuf.columns:
    print(c)

DESYNPUF_ID
BENE_BIRTH_DT
BENE_DEATH_DT
SP_STATE_CODE
BENE_COUNTY_CD
BENE_HI_CVRAGE_TOT_MONS
BENE_SMI_CVRAGE_TOT_MONS
BENE_HMO_CVRAGE_TOT_MONS
PLAN_CVRG_MOS_NUM
SP_ALZHDMTA
SP_CHF
SP_CHRNKIDN
SP_CNCR
SP_COPD
SP_DEPRESSN
SP_DIABETES
SP_ISCHMCHT
SP_OSTEOPRS
SP_RA_OA
SP_STRKETIA
MEDREIMB_IP
BENRES_IP
PPPYMT_IP
MEDREIMB_OP
BENRES_OP
PPPYMT_OP
MEDREIMB_CAR
BENRES_CAR
PPPYMT_CAR
FIPS_COUNTY_CODE
FULL_FIPS_CODE
CBSA
CBSA_NAME
FIPS_STATE_CODE
CLM_ID
SEGMENT
CLM_FROM_DT
CLM_THRU_DT
PRVDR_NUM
CLM_PMT_AMT
NCH_PRMRY_PYR_CLM_PD_AMT
AT_PHYSN_NPI
OP_PHYSN_NPI
OT_PHYSN_NPI
CLM_ADMSN_DT
ADMTNG_ICD9_DGNS_CD
CLM_PASS_THRU_PER_DIEM_AMT
NCH_BENE_IP_DDCTBL_AMT
NCH_BENE_PTB_DDCTBL_AMT
NCH_BENE_PTA_COINSRNC_LBLTY_AM
NCH_BENE_PTB_COINSRNC_AMT
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM
CLM_UTLZTN_DAY_CNT
NCH_BENE_DSCHRG_DT
CLM_DRG_CD
ICD9_DGNS_CD_1
ICD9_DGNS_CD_2
ICD9_DGNS_CD_3
ICD9_DGNS_CD_4
ICD9_DGNS_CD_5
ICD9_DGNS_CD_6
ICD9_DGNS_CD_7
ICD9_DGNS_CD_8
ICD9_DGNS_CD_9
ICD9_DGNS_CD_10
ICD9_PRCDR_CD_1
ICD9_PRCDR_CD_2
ICD9_PRCD

In [36]:
desynpuf["TOTCHRONIC"] = desynpuf["B_ALZHDMTA"] + desynpuf["B_CHF"] + desynpuf["B_CNCR"] + desynpuf["B_COPD"] + desynpuf["B_DEPRESSN"] + desynpuf["B_DIABETES"] + desynpuf["B_ISCHMCHT"] + desynpuf["B_OSTEOPRS"] + desynpuf["B_CHRNKIDN"] + desynpuf["B_RA_OA"] + desynpuf["B_STRKETIA"]
desynpuf["TOTIPCOST"] = desynpuf["MEDREIMB_IP"] + desynpuf["BENRES_IP"] + desynpuf["PPPYMT_IP"]

#In this case, since we are only looking at Inpatient Claims for 2009, ALLCOSTS will be the same as TOTIPCOST
desynpuf["ALLCOSTS"] = desynpuf["TOTIPCOST"]

In [37]:
# Combine Coverage Data and Number of Providers
desynpuf["CVRG_MOS"] = desynpuf["BENE_HI_CVRAGE_TOT_MONS"].copy()
desynpuf["RX_CVRG_MOS"] = desynpuf["PLAN_CVRG_MOS_NUM"].copy()
desynpuf["LENGTH_OF_STAY"] = desynpuf["CLM_UTLZTN_DAY_CNT"].copy()

# Drop old condition indicators
desynpuf.drop(["SP_ALZHDMTA", "SP_CHF", "SP_CHRNKIDN", "SP_CNCR", "SP_COPD", "SP_DEPRESSN", "SP_DIABETES", "SP_ISCHMCHT",
       "SP_OSTEOPRS", "SP_RA_OA", "SP_STRKETIA"], axis=1, inplace=True)

# Drop outpatient related fields (not using), and YEAR (not using)
desynpuf.drop(["MEDREIMB_OP", "BENRES_OP", "PPPYMT_OP", "MEDREIMB_CAR", "BENRES_CAR", "PPPYMT_CAR", "YEAR",
       "NCH_PRMRY_PYR_CLM_PD_AMT", "NCH_BENE_PTB_DDCTBL_AMT", "NCH_BENE_PTB_COINSRNC_AMT", 
        "OP_PHYSN_NPI", "OT_PHYSN_NPI"], axis=1, inplace=True)

desynpuf.drop(["BENE_HI_CVRAGE_TOT_MONS", "BENE_SMI_CVRAGE_TOT_MONS", "BENE_HMO_CVRAGE_TOT_MONS", 
               "PLAN_CVRG_MOS_NUM"], axis=1, inplace=True)

In [38]:
# Drop diagnosis and HCPCS codes - we've already used them for Elix calculation and counting # of codes and procedures

desynpuf.drop(["ICD9_DGNS_CD_1","ICD9_DGNS_CD_2","ICD9_DGNS_CD_3","ICD9_DGNS_CD_4","ICD9_DGNS_CD_5",
                  "ICD9_DGNS_CD_6","ICD9_DGNS_CD_7","ICD9_DGNS_CD_8","ICD9_DGNS_CD_9","ICD9_DGNS_CD_10",
                  "ICD9_PRCDR_CD_1","ICD9_PRCDR_CD_2","ICD9_PRCDR_CD_3","ICD9_PRCDR_CD_4","ICD9_PRCDR_CD_5",
                  "ICD9_PRCDR_CD_6"], axis=1, inplace=True)

desynpuf.drop(["HCPCS_CD_1", "HCPCS_CD_2","HCPCS_CD_3","HCPCS_CD_4","HCPCS_CD_5","HCPCS_CD_6","HCPCS_CD_7",
                      "HCPCS_CD_8","HCPCS_CD_9","HCPCS_CD_10","HCPCS_CD_11","HCPCS_CD_12","HCPCS_CD_13",
                      "HCPCS_CD_14","HCPCS_CD_15","HCPCS_CD_16","HCPCS_CD_17","HCPCS_CD_18","HCPCS_CD_19",
                      "HCPCS_CD_20","HCPCS_CD_21","HCPCS_CD_22","HCPCS_CD_23","HCPCS_CD_24","HCPCS_CD_25",
                      "HCPCS_CD_26","HCPCS_CD_27","HCPCS_CD_28","HCPCS_CD_29","HCPCS_CD_30","HCPCS_CD_31",
                      "HCPCS_CD_32","HCPCS_CD_33","HCPCS_CD_34","HCPCS_CD_35","HCPCS_CD_36","HCPCS_CD_37",
                      "HCPCS_CD_38","HCPCS_CD_39","HCPCS_CD_40","HCPCS_CD_41","HCPCS_CD_42","HCPCS_CD_43",
                      "HCPCS_CD_44","HCPCS_CD_45"], axis=1, inplace=True)

In [39]:
# There are no procedures for this data set, so there is no need to keep this field (calculated earlier)

desynpuf.drop(["HCPCS_COUNT"], axis=1, inplace=True)

In [40]:
# Count physicians (for inpatient visits, use PRVDR_NUM and AT_PHYSN_NPI)
phys = desynpuf[["PRVDR_NUM","AT_PHYSN_NPI"]]

desynpuf["PHYS_COUNT"] = phys.count(axis=1)
# do not fill NaN values - NaNs are appropriate for patients without claims.

In [41]:
desynpuf.isnull().sum() 

DESYNPUF_ID                            0
BENE_BIRTH_DT                          0
BENE_DEATH_DT                     118801
SP_STATE_CODE                          0
BENE_COUNTY_CD                         0
MEDREIMB_IP                            0
BENRES_IP                              0
PPPYMT_IP                              0
FIPS_COUNTY_CODE                    1817
FULL_FIPS_CODE                      1817
CBSA                               27860
CBSA_NAME                          27860
FIPS_STATE_CODE                     1817
CLM_ID                             95832
SEGMENT                            95832
CLM_FROM_DT                        95832
CLM_THRU_DT                        95832
PRVDR_NUM                          95832
CLM_PMT_AMT                        95832
AT_PHYSN_NPI                       95903
CLM_ADMSN_DT                       95832
ADMTNG_ICD9_DGNS_CD                96025
CLM_PASS_THRU_PER_DIEM_AMT         95832
NCH_BENE_IP_DDCTBL_AMT             96414
NCH_BENE_PTA_COI

In [42]:
# Remove 82 rows with no Elixhauser scores

desynpuf = desynpuf[~desynpuf["MEAN_ELIX_SCORE"].isnull()]

In [43]:
# Explore the issue of 1,817 rows without FIPS codes

no_fips = desynpuf[desynpuf["FULL_FIPS_CODE"].isnull()]
no_fips[["FULL_FIPS_CODE", "CBSA", "SP_STATE_CODE", "BENE_COUNTY_CD"]]

,FULL_FIPS_CODE,CBSA,SP_STATE_CODE,BENE_COUNTY_CD
43,NaN,NaN,54,390
77,NaN,NaN,54,630
113,NaN,NaN,54,999
177,NaN,NaN,54,130
197,NaN,NaN,54,400
...,...,...,...,...
120550,NaN,NaN,54,1
120554,NaN,NaN,54,10
120594,NaN,NaN,54,1
120613,NaN,NaN,54,150


It appears these are beneficiaries who live outside the U.S. 

When SSA State Code > 53, these numbers correspond to places outside the U.S.

For example, Code 54 is "Africa".

I didn't know there was such a thing, but apparently there is!

So we will keep these rows, but set all their FIPS codes to zeros.

In [44]:
desynpuf.loc[desynpuf["FULL_FIPS_CODE"].isnull(), "FULL_FIPS_CODE"] = "0"
desynpuf.loc[desynpuf["FULL_FIPS_CODE"].isnull(), "FIPS_COUNTY_CODE"] = "0"
desynpuf.loc[desynpuf["FULL_FIPS_CODE"].isnull(), "FIPS_STATE_CODE"] = "0"

desynpuf.drop(["SP_STATE_CODE", "BENE_COUNTY_CD"], axis=1, inplace=True)

desynpuf.isnull().sum() 

DESYNPUF_ID                            0
BENE_BIRTH_DT                          0
BENE_DEATH_DT                     118720
MEDREIMB_IP                            0
BENRES_IP                              0
PPPYMT_IP                              0
FIPS_COUNTY_CODE                    1817
FULL_FIPS_CODE                         0
CBSA                               27839
CBSA_NAME                          27839
FIPS_STATE_CODE                     1817
CLM_ID                             95763
SEGMENT                            95763
CLM_FROM_DT                        95763
CLM_THRU_DT                        95763
PRVDR_NUM                          95763
CLM_PMT_AMT                        95763
AT_PHYSN_NPI                       95833
CLM_ADMSN_DT                       95763
ADMTNG_ICD9_DGNS_CD                95956
CLM_PASS_THRU_PER_DIEM_AMT         95763
NCH_BENE_IP_DDCTBL_AMT             96345
NCH_BENE_PTA_COINSRNC_LBLTY_AM     95763
NCH_BENE_BLOOD_DDCTBL_LBLTY_AM     95763
CLM_UTLZTN_DAY_C

In [45]:
desynpuf.shape

(120623, 57)

## Beneficiaries with multiple claims & beneficiaries who died in 2009

Out of 120,623 rows, there are 1,903 where beneficiary death date is filled in.

The 120,623 rows contain 95,763 beneficiaries with no claims that year and 18,617 beneficiaries with at least 1 claim.

In [46]:
temp = desynpuf.groupby(["DESYNPUF_ID"]).aggregate({"CLM_ID":"count"})

print("There are", temp[temp["CLM_ID"]>0].shape[0], "beneficiaries with 1 or more inpatient claims in 2009.\n")
print("There are", temp[temp["CLM_ID"]==0].shape[0], "beneficiaries with no claims in 2009.\n")
print("There are", temp[temp["CLM_ID"]>5].shape[0], "beneficiaries with more than 5 claims in 2009.\n")

There are 18617 beneficiaries with 1 or more inpatient claims in 2009.

There are 95763 beneficiaries with no claims in 2009.

There are 24 beneficiaries with more than 5 claims in 2009.



## Columns that will not be used

A large # of these columns will neither be used in the RAM, nor Correspondence Analysis.

These can be dropped:

SP_ESRD (already converted to B_ESRD), TOTIPCOST (repetitive, since this is already only IP costs for 2009), MEAN_ELIX_GRP_TOT (don't need both this and the ELIX_SCORE), NCH_BENE_DSCHRG_DT, CLM_UTLZTN_DAY_CNT (already converted to LENGTH_OF_STAY),NCH_BENE_BLOOD_DDCTBL_LBLTY_AM, NCH_BENE_PTA_COINSRNC_LBLTY_AM, NCH_BENE_IP_DDCTBL_AMT, CLM_PASS_THRU_PER_DIEM_AMT, ADMTNG_ICD9_DGNS_CD, CLM_ADMSN_DT, AT_PHYSN_NPI, CLM_PMT_AMT,
PRVDR_NUM, CLM_THRU_DT, CLM_FROM_DT, CLM_FROM_DT, SEGMENT, CLM_ID, PPPYMT_IP, BENRES_IP, MEDREIMB_IP

In [51]:
# Drop unused columns 

desynpuf.drop(["SP_ESRD", "TOTIPCOST", "MEAN_ELIX_GRP_TOT", "NCH_BENE_DSCHRG_DT", "CLM_UTLZTN_DAY_CNT",
               "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM", "NCH_BENE_PTA_COINSRNC_LBLTY_AM", "NCH_BENE_IP_DDCTBL_AMT", 
               "CLM_PASS_THRU_PER_DIEM_AMT", "ADMTNG_ICD9_DGNS_CD", "CLM_ADMSN_DT", "AT_PHYSN_NPI", "CLM_PMT_AMT",
               "PRVDR_NUM","CLM_THRU_DT", "CLM_FROM_DT", "CLM_FROM_DT", "SEGMENT", "PPPYMT_IP", "BENRES_IP", 
               "MEDREIMB_IP"], axis=1, inplace=True)


In [52]:
desynpuf.shape

(120623, 37)

In [53]:
# Fix datetime formats

desynpuf["BENE_BIRTH_DT"] = pd.to_datetime(null_age["BENE_BIRTH_DT"], format="%Y-%m-%d")
desynpuf["BENE_DEATH_DT"] = pd.to_datetime(null_age["BENE_DEATH_DT"], format="%Y-%m-%d")

desynpuf.loc[(~desynpuf["BENE_DEATH_DT"].isnull()), "B_DIED"] = 1
desynpuf.loc[(desynpuf["BENE_DEATH_DT"].isnull()), "B_DIED"] = 0



In [54]:
desynpuf["CBSA"].fillna(0, inplace=True)
desynpuf["FIPS_STATE_CODE"].fillna(0, inplace=True)
desynpuf["FIPS_COUNTY_CODE"].fillna(0, inplace=True)
desynpuf["FULL_FIPS_CODE"].fillna(0, inplace=True)
desynpuf["CBSA"] = desynpuf["CBSA"].astype(int)
desynpuf["FIPS_STATE_CODE"] = desynpuf["FIPS_STATE_CODE"].astype(int)
desynpuf["FIPS_COUNTY_CODE"] = desynpuf["FIPS_COUNTY_CODE"].astype(int)
desynpuf["FULL_FIPS_CODE"] = desynpuf["FULL_FIPS_CODE"].astype(int)

In [55]:
#agg = desynpuf.groupby(["DESYNPUF_ID"]).aggregate({"BENE_AGE":"mean",
#                                                   "B_SEX":"mean",
#                                                   "NH_WHITE": "mean",
#                                                   "AA_BLACK" : "mean",
#                                                   "OTHER": "mean",
#                                                   "NW_HISPANIC": "mean",
#                                                    "B_ALZHDMTA":"sum",
#                                                    "B_CHF":"sum" ,
#                                                    "B_CNCR":"sum",
#                                                    "B_COPD":"sum",
#                                                    "B_DEPRESSN":"sum",
#                                                    "B_DIABETES":"sum",
#                                                    "B_ISCHMCHT":"sum",
#                                                    "B_OSTEOPRS":"sum",
#                                                    "B_CHRNKIDN":"sum",
#                                                    "B_RA_OA":"sum",
#                                                    "B_STRKETIA":"sum",
#                                                    "B_ESRD":"sum",
#                                                    "B_DIED":"sum",
#                                                    "CVRG_MOS":"mean",
#                                                    "RX_CVRG_MOS":"mean",
#                                                    "MEAN_ELIX_SCORE":"mean",
#                                                    "CLM_ID":"count",
#                                                     "CODES_COUNT":["sum","mean"],
#                                                     "TOTCHRONIC":"mean",
#                                                     "ALLCOSTS":["sum","mean"],
#                                                     "LENGTH_OF_STAY":["sum","mean"],
#                                                     "PHYS_COUNT":["sum","mean"],
#                                                     "CBSA":"mean",
#                                                     "FULL_FIPS_CODE":"mean",
#                                                     "FIPS_STATE_CODE":"mean",
#                                                     "FIPS_COUNTY_CODE":"mean"})


In [56]:
#agg.columns = ["BENE_AGE", "B_SEX", "NH_WHITE", "AA_BLACK", "OTHER", "NW_HISPANIC", 
#               "B_ALZHDMTA", "B_CHF", "B_CNCR", "B_COPD", "B_DEPRESSN", "B_DIABETES", "B_ISCHMCHT", 
#               "B_OSTEOPRS", "B_CHRNKIDN", "B_RA_OA", "B_STRKETIA", "B_ESRD", "B_DIED", "CVRG_MOS", "RX_CVRG_MOS",
#               "MEAN_ELIX_SCORE", "TOTAL_VISITS", "TOTAL_CODES", "MEAN_CODES_VISIT", "MEAN_CHRONIC", 
#               "ALLCOSTS", "MEAN_COST_VISIT", "TOTAL_LOS", "MEAN_LOS", "TOTAL_PHYS", "MEAN_PHYS_VISIT",
#              "CBSA", "FULL_FIPS_CODE", "FIPS_STATE_CODE", "FIPS_COUNTY_CODE"]

In [57]:
#agg.loc[agg["B_ALZHDMTA"]>1, "B_ALZHDMTA"] = 1
#agg.loc[agg["B_CHF"]>1, "B_CHF"] = 1
#agg.loc[agg["B_CNCR"]>1, "B_CNCR"] = 1
#agg.loc[agg["B_COPD"]>1, "B_COPD"] = 1
#agg.loc[agg["B_DEPRESSN"]>1, "B_DEPRESSN"] = 1
#agg.loc[agg["B_DIABETES"]>1, "B_DIABETES"] = 1
#agg.loc[agg["B_ISCHMCHT"]>1, "B_ISCHMCHT"] = 1
#agg.loc[agg["B_OSTEOPRS"]>1, "B_OSTEOPRS"] = 1
#agg.loc[agg["B_CHRNKIDN"]>1, "B_CHRNKIDN"] = 1
#agg.loc[agg["B_RA_OA"]>1, "B_RA_OA"] = 1
#agg.loc[agg["B_STRKETIA"]>1, "B_STRKETIA"] = 1
#agg.loc[agg["B_ESRD"]>1, "B_ESRD"] = 1

#agg["TOTCHRONIC"] = agg["B_ALZHDMTA"] + agg["B_CHF"] + agg["B_CNCR"] + agg["B_COPD"] + agg["B_DEPRESSN"] + agg["B_DIABETES"] + agg["B_ISCHMCHT"] + agg["B_OSTEOPRS"] + agg["B_CHRNKIDN"] + agg["B_RA_OA"] + agg["B_STRKETIA"]

In [58]:
#agg.loc[((agg["BENE_AGE"]>0) & (agg["BENE_AGE"]<19)), "AGE_GROUP"] = "0-18"
#agg.loc[((agg["BENE_AGE"]>18) & (agg["BENE_AGE"]<45)), "AGE_GROUP"] = "19-44"
#agg.loc[((agg["BENE_AGE"]>44) & (agg["BENE_AGE"]<65)), "AGE_GROUP"] = "45-64"
#agg.loc[((agg["BENE_AGE"]>64) & (agg["BENE_AGE"]<75)), "AGE_GROUP"] = "65-74"
#agg.loc[((agg["BENE_AGE"]>74) & (agg["BENE_AGE"]<85)), "AGE_GROUP"] = "75-84"
#agg.loc[(agg["BENE_AGE"]>84), "AGE_GROUP"] = "85+"

In [60]:
#agg.shape

In [61]:
#agg.info()

In [62]:
SUM_TOTAL = desynpuf["ALLCOSTS"].sum()
desynpuf["PERCENT_COSTS"] = desynpuf["ALLCOSTS"] / SUM_TOTAL * 100

In [64]:
desynpuf["CLM_ID"] = desynpuf["CLM_ID"].astype(str)

In [65]:
desynpuf.head(30)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,FIPS_COUNTY_CODE,FULL_FIPS_CODE,CBSA,CBSA_NAME,FIPS_STATE_CODE,CLM_ID,CLM_DRG_CD,...,MEAN_ELIX_SCORE,CODES_COUNT,TOTCHRONIC,ALLCOSTS,CVRG_MOS,RX_CVRG_MOS,LENGTH_OF_STAY,PHYS_COUNT,B_DIED,PERCENT_COSTS
0,0000438E79D01BEA,1937-05-01,NaT,57,12057,45300,"Tampa-St. Petersburg-Clearwater, FL",12,nan,NaN,...,0.000000,0,0,0.0,12,12,NaN,0,0.0,0.000000
1,00010E7BEA69142C,1938-04-01,2009-11-01,3,10003,48864,"Wilmington, DE-MD-NJ",10,nan,NaN,...,1.000000,0,2,0.0,12,12,NaN,0,1.0,0.000000
2,00015BF6509E1DF7,1934-05-01,NaT,119,37119,16740,"Charlotte-Gastonia-Concord, NC-SC",37,nan,NaN,...,0.000000,0,3,0.0,12,12,NaN,0,0.0,0.000000
3,000240D599ED789C,NaT,NaT,35,55035,20740,"Eau Claire, WI",55,692591107784071.0,488,...,2.000000,15,5,24068.0,12,12,12.0,2,0.0,0.004938
4,0003E73AFBA840A8,1941-09-01,NaT,55,36055,40380,"Rochester, NY",36,nan,NaN,...,0.000000,0,0,0.0,0,0,NaN,0,0.0,0.000000
5,00040FBA47F90D4F,1924-12-01,NaT,35,34035,20764,"Edison-New Brunswick, NJ",34,nan,NaN,...,0.800000,0,2,0.0,12,12,NaN,0,0.0,0.000000
6,000588F33B7B0753,NaT,NaT,113,19113,16300,"Cedar Rapids, IA",19,692071107770218.0,207,...,2.000000,10,6,20504.0,12,3,6.0,2,0.0,0.004207
7,000588F33B7B0753,NaT,NaT,113,19113,16300,"Cedar Rapids, IA",19,692401107764276.0,619,...,2.000000,5,6,20504.0,12,3,1.0,2,0.0,0.004207
8,000588F33B7B0753,NaT,NaT,113,19113,16300,"Cedar Rapids, IA",19,692631107760334.0,180,...,2.000000,4,6,20504.0,12,3,2.0,2,0.0,0.004207
9,0006071C876CEADF,1931-09-01,NaT,141,48141,21340,"El Paso, TX",48,nan,NaN,...,0.000000,0,4,0.0,12,12,NaN,0,0.0,0.000000


In [66]:
desynpuf.to_csv("/home/kylier/python/DS/data/PUF/PUF_ELIX_ALLIP2009.csv")